In [18]:
import numpy as np
import pickle
from tqdm import tqdm, tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors

import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [19]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-mobilenet.pickle',
                                'rb'))
class_ids = pickle.load(open('data/class_ids-caltech101.pickle', 'rb'))

In [20]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  6411
Number of features per image =  1024


In [21]:
random_image_index = random.randint(0, num_images)

Brute force nearest neighbour on one image

In [10]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)

31.9 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

76.7 ms ± 7.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


k-d Tree Algorithm on one image

In [8]:

%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)

2.79 s ± 216 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

14.7 ms ± 992 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ball Tree Algorithm on one image

In [12]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)

1.95 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:

%timeit neighbors.kneighbors([feature_list[random_image_index]])

11.4 ms ± 951 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:

random_image_indices = random.sample(range(0, num_images), 1000)
random_feature_list = [
    feature_list[each_index] for each_index in random_image_indices
]

 Brute Force Algorithm on a set of images

In [13]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)
%timeit neighbors.kneighbors(feature_list)

1.77 s ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


 k-d Tree Algorithm on a set of images

In [16]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

14.3 s ± 368 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


 Ball Tree Algorithm on a set of images

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

# PCA

In [22]:
num_feature_dimensions = 100
num_feature_dimensions = min(num_images, num_feature_dimensions,
                             len(feature_list[0]))

train pca

In [23]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()

In [24]:
print(pca.explained_variance_ratio_[0:20])

[0.04808487 0.03902655 0.03520469 0.0229082  0.01972816 0.01739131
 0.01555806 0.01291987 0.01241799 0.01133363 0.01028689 0.00926208
 0.00849703 0.00841846 0.00785455 0.00749922 0.00687655 0.00677042
 0.00629248 0.00604648]


PCA + Brute Force Algorithm on one image

In [20]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)

30.7 ms ± 3.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])


2.88 ms ± 580 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


PCA + k-d Tree Algorithm on one image

In [23]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)

194 ms ± 8.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])


1.81 ms ± 82.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


PCA + Ball Tree Algorithm on one image

In [25]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)

117 ms ± 8.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

1.47 ms ± 84 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
random_image_indices = random.sample(range(0, num_images), 1000)
random_feature_list_compressed = [
    feature_list_compressed[each_index] for each_index in random_image_indices
]

PCA + Brute Force Algorithm on a set of images

In [11]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)
%timeit neighbors.kneighbors(feature_list_compressed)

1.57 s ± 69.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


PCA + k-d Tree Algorithm on a set of images

In [12]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(feature_list_compressed)

11.1 s ± 273 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


PCA + Ball Tree Algorithm on a set of images

In [33]:
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1.62 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Annoy

In [2]:
!pip install annoy

     |████████████████████████████████| 644 kB 1.9 MB/s eta 0:00:01
  Created wheel for annoy: filename=annoy-1.16.3-cp37-cp37m-macosx_10_7_x86_64.whl size=64936 sha256=655792597989fdabd6fbc3d1867af30db09fb0c449b0b6d69af0b9e0877a0aec
  Stored in directory: /Users/udaykumar/Library/Caches/pip/wheels/39/36/d4/ee348a7240ca3e8d1fcbf04ebe46d45f2879ccb094a40f5706
Successfully built annoy


In [25]:
from annoy import AnnoyIndex

In [26]:
# Time the indexing for Annoy
t = AnnoyIndex(100)  # Length of item vector that will be indexed
starttime = time.time()
for i in range(num_images-1):
    feature = feature_list_compressed[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


0.022800207138061523


True

Annoy on one image

In [27]:
u = AnnoyIndex(100)
%timeit u.get_nns_by_vector(feature_list_compressed[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                              5,
                              include_distances=True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


2.91 µs ± 16.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [28]:
def calculate_annoy_time():
    for i in range(0, 1000):
        indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                                      5,
                                      include_distances=True)

Annoy on a set of images

In [29]:
%timeit calculate_annoy_time()

3.42 ms ± 587 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Falconn

In [36]:
import falconn

In [37]:
# Setup different parameters for Falonn
parameters = falconn.LSHConstructionParameters()
num_tables = 1
parameters.l = num_tables
parameters.dimension = num_feature_dimensions
parameters.distance_function = falconn.DistanceFunction.EuclideanSquared
parameters.lsh_family = falconn.LSHFamily.CrossPolytope
parameters.num_rotations = 1
parameters.num_setup_threads = 1
parameters.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable

# Train the Falconn model
falconn.compute_number_of_hash_functions(16, parameters)

On One Image falconn

In [38]:
dataset = np.array(feature_list_compressed)
a = np.random.randn(8677, 100)
a /= np.linalg.norm(a, axis=1).reshape(-1, 1)
dataset = a

index = falconn.LSHIndex(parameters)
%time index.setup(dataset)

query_object = index.construct_query_object()
num_probes = 1
query_object.set_num_probes(num_probes)

searchQuery = np.array(feature_list_compressed[random_image_index])
searchQuery = a[0]
%timeit query_object.find_k_nearest_neighbors(searchQuery, 5)

CPU times: user 20.8 ms, sys: 369 µs, total: 21.2 ms
Wall time: 10.8 ms
6.12 µs ± 477 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


On set of images

In [74]:
searchQuery = np.array(feature_list_compressed)
searchQuery = a[0]
%timeit query_object.find_k_nearest_neighbors(searchQuery, 5)

58.7 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# NMSLib

In [39]:
import nmslib


In [40]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(feature_list_compressed)
index.createIndex({'post': 2}, print_progress=True)

on One Image nmslib

In [45]:
# Query for the nearest neighbors of the first datapoint
%timeit index.knnQuery(feature_list_compressed[random_image_index], k=5)
ids, distances = index.knnQuery(feature_list_compressed[random_image_index],
                                k=5)

194 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


On set of Images nmslib

In [43]:
# Get all nearest neighbors for all the datapoint
# using a pool of 4 threads to compute
%timeit index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)
neighbors = index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)

99.6 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# LSH

In [67]:
## Loading Feature dictionary
feature_dict = pickle.load(open('data/feature_dict.p','rb'))
lsh = pickle.load(open('data/lsh.p','rb'))


on one image

In [75]:
%timeit lsh.query(feature_dict[list(feature_dict.keys())[random_image_index]].flatten(),num_results=5, distance_func='hamming')

1.79 s ± 74.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


on set of images

In [78]:
def calculate_lsh_time():
    for i in range(0, 10):
        response = lsh.query(feature_dict[list(feature_dict.keys())[random_image_index]].flatten(),num_results=5, distance_func='hamming')

In [80]:
%timeit calculate_lsh_time()

22.9 s ± 1.73 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
